In [ ]:
%run -i underthecovers/python/common.py
%run -i underthecovers/python/ln_preamble.py


# Status

## Problems

1. Very poor and inconsistent performance on operate-first
    - discussed this with RedHat performance seems to be getting better
2. nootbook output cells data is painful in git consider other options:
    - In some sense this is what Juptext is supposed to help with .... but I could not figure out the right way to get jupyter-book to recongnize Juptext markdown documents to be executable and hence they did not have the launch icon generated for them.
    - in the end I have settled on using [nbstripout](https://github.com/kynan/nbstripout) ... container has a default install of it configured for all users ... see `/etc/gitconfig` 
3. Docker Image is very large
    - have not been worried about size
        - broken down into 3 docker stages:
            1. base  
                - From jupyter/minimal-notebook
                    - Installs a bunch of ubuntu packages to make generic book development and teaching easier 
                    - includes a build of a recent version of gdb as distros don't seem to include one yet
            2. base-unmin
                - From base (above)
                    - invokes unminize command so that user experience is more like a full blown system.  Eg manpages and other doc are present
            3. bu-cs-book-dev
                - From base-unmin
                    - adds all the extra python packages for the jupyter-book and rise stuff I depend on for UC:SLS
                    - adds custom startup scripts so that starting on MOC/operate-first behaves correctly
                    

## Next Steps

1. Work on improving Dockerfiles to avoid unnecessary layers
2. Exploit startup hooks to try and customize the machine a bit more:
    - add correct user name to /etc/passwd
    - see if we can figure out a way to rename home dir
    - fix env then USER and HOME, etc
    - fix hostname
3. document that we are using the default jupyter user jovyan (https://jupyter.readthedocs.io/en/latest/community/content-community.html#what-is-a-jovyan) to be our container user
4. Add ability to communicate between shell/terminal and python.  
    - useful things to keep in mind : https://askubuntu.com/questions/853102/how-do-i-run-a-command-before-or-after-every-command-written-in-the-console
    - socat as a man in the middle
    - named pipes
    - background python threads
    

## Things to think about using

- https://towardsdatascience.com/7-essential-tips-for-writing-with-jupyter-notebook-60972a1a8901#231d
-

## Working  
0.  This url works locally if you have your jupyter server running locally:<br> (eg make nb on container repo)
    http://127.0.0.1:8888/git-pull?bu-cs-jupyter-book%3Alatest=&repo=https%3A%2F%2Fgithub.com%2Fjappavoo%2FUndertheCovers&urlpath=tree%2FUndertheCovers%2Funderthecovers%2F&branch=main
1.  Operate First direct spawn url for bu-cs-jupyter-book:latest:<br> 
https://jupyterhub-opf-jupyterhub.apps.smaug.na.operate-first.cloud/hub/spawn?bu-cs-jupyter-book:latest
2. To get nbgitpuller to clone or sync the UndertheCovers repo:<br>
https://jupyterhub-opf-jupyterhub.apps.smaug.na.operate-first.cloud/hub/user-redirect/git-pull?bu-cs-jupyter-book%3Alatest=&repo=https%3A%2F%2Fgithub.com%2Fjappavoo%2FUndertheCovers&urlpath=tree%2FUndertheCovers%2Funderthecovers%2FL00_210_JA.ipynb&branch=main
  - Other very usefull urls:
      - authenticated users can see the Jupyterhub namespace on openshift here:<br>
https://jupyterhub-opf-jupyterhub.apps.smaug.na.operate-first.cloud/k8s/ns/opf-jupyterhub/pods <br>
      - Your pod can be found here: <br>
https://jupyterhub-opf-jupyterhub.apps.smaug.na.operate-first.cloud/k8s/ns/opf-jupyterhub/pods/jupyterhub-nb-jappavoo-40bu-2eedu <br>
This can give you a bit more info on the state of your pod
3. 2 required futzing with the image so that the group id of the joyvan home directory is root (see [dockerfile](https://github.com/jappavoo/bu-cs-book-dev/blob/c88bc65d65b334150724981dad145f395e177505/Dockerfile#L87)): <br>
``` shell
# as a hack we are going to try changing group id of /home/joyvan to be root to see if I can trick things into
# working on the moc
RUN chgrp -R root /home/jovyan
```
The startup logic of the container which we inherited from the jupyter/minimal-notebook:latest (https://github.com/jupyter/docker-stacks/blob/master/base-notebook/start.sh) does a bunch magic:
  - when spawned by the main jupyterhub server on oprate-first the container seems to launch with a userid of the operate-first uesr (eg in my case 1000950000) and a gid of root (0)
  - which does not match the uid of the joyvan user in the container (1000) so the startup script replaces the joyvan user with a new joyvan user who's uid is the operate-first user id.  
  - however it does nothing to the permissions of the homedir and it seems to let the gid of root persist
  - so by default the home directory is readable but can not be written
  - to fix this I modify the container's base permission of the joyvan homedir to be group 0
  - oddly enough this all seems to work
4. Oddly enough this also seems to solve my problem of persistent state:  The default logic is starting the jupyter server in the joyvan home dir which is not on the volume mounted by operate-first and its contents is server instance specific.  Destroying the server (stopping it) and restarting seeds the joyvan home dir back to its initial state from the container
5. To access the api we must have the api token this however is not present in the same way as it was when running the jupyter notebook server locally -- see ln_preamble.py:<br>
   on operate-first jupyerhub the api token is in an environment variable
6. To make get requests we must use a full url to the localhost introduced localhost_url in ln_preamble
7. To access the users server urls we need to prefix with `/hub/user-redirect` or any equivalent eg. to get access terminal 3 it would be `/hub/user-redirect`.  So now to embed a terminal we would can succesfully do the following: <br>
``` python
IFrame('/hub/user-redirect/terminals/2', 1400, 600)
```
or
``` python
IFrame('/user/jappavoo@bu.edu/terminals/2', 1400, 600)
```
or
``` python
IFrame('/hub/user/jappavoo@bu.edu/terminals/2', 1400, 600)
```
which means that we can use base_url as reported by server info in both environments to embedded a terminal
``` python
IFrame(base_url + 'terminals/2', 1400, 600)
```
The above now lets us embed terminals in the notebooks similar to the functionality of running a locally hosted version of the container and starting the jupyter notebook server by hand (see make devnb).  Have rewriten python/ln_preamble.py appropriately.
8. Had a very hard time with getting python package installed and configure in a way that worked on opf with all constraints meet.  The current head of the bu-cs-book-dev repo has the version that I found to work:  
    - boots on opf (from some reason prior commit in which I very carefully tested python package commands fails to boot).  
    - all jupyter notebook extensions that I want installed by default work and are on when user opens a notebook
    - the right version of jupyter books is installed -- use of conda got the wrong version switch to using pip install and this got the right version
9. cleanup home dir and added stuff to bash rc to have better default behaviour


## TIPS and TRICKS

- to remove and hide cells from jupyter book -  add the following tags to the meta data of the cell: `remove-input, remove-output, remove-cell, hide-input, hide-output, hide-cell` see [jupyter-book doc](https://jupyterbook.org/interactive/hiding.html#removing-code-cell-content)
- to hide from rise use skip slide tag
- see [example](#splitColumn) of split column extension for two column layout rather than using html
- see my [displayBytes](#displayBytes) below for better binary data support through panda tables
- see my [htmlFig](#htmlFig) below for a figure examples
- see my [TermShellCmd](#TermShellCmd) below for running shell commands and showing their output in a nice way... **This is becoming out of date.  See the bash and tty session stuff below**
- see my [TtySession](#TtySession) below for a a more interesting way for running terminal based examples
- see my [MDBox](#MDBox) and friends for creating scrolled output content that is really usefull when mixing with terminals 

<a id='splitColumn'></a>
# Better two split column html
- from https://en.wikipedia.org/wiki/VT220

## Manual html Figures
Some html for better figures.  Here is a single figure that spans the full column
<table id="fig:1Term" align="center" width="100%" cellpadding="0" cellspacing="0" border="0" style="border-collapse: collapse; margin: auto 2em auto auto" >
    <tr style="padding: 0; margin: 0;">
        <td style="padding: 0; margin: 0; background-color: white;">
            <div style="margin-right: auto; margin-left: auto; padding: 0; margin: 0;">
              <figure style="width:100%; padding: 0; margin: 0;border: 1px solid red;" >
                  <img src="underthecovers/images/shell.jpg" width="100%" style="padding: 0; margin: 0;">
                   <div align="right" style="color: #978282; line-height: 0; font-size: 2vmin">
                    <em>
                      See <a href="https://en.wikipedia.org/wiki/VT220">vt220</a>
                    </em>
                  </div>
                  <figcaption>
                    <div style="background-color: red; margin-right:auto; margin-left:auto; text-align: center;">
                       <i style="color: white">B: Terminal 2</i>
                    </div>
                  </figcaption>
                </figure>
            </div>
        </td>
    </tr>
    <caption align="bottom" style="text-align: left; padding: 0; margin: 0;" >
          <i>Figure 1: An <a href="https://en.wikipedia.org/wiki/VT220">vt220</a> ASCII Temrinal</i> 
    </caption>
</table>

In the <a href="#fig:1Term">figure</a> we see a classic ASCII terminal.

<table id="fig:1Term-2" align="right" width="50%" cellpadding="0" cellspacing="0" border="0" style="border-collapse: collapse; margin: auto auto auto .5 em" >
    <tr style="padding: 0; margin: 0;">
        <td style="padding: 0; margin: 0; background-color: white;">
            <div style="margin-right: auto; margin-left: auto; padding: 0; margin: 0;">
              <figure style="width:100%; padding: 0; margin: 0;border: 1px solid red;" >
                  <img src="underthecovers/images/shell.jpg" width="100%" style="padding: 0; margin: 0;">
                   <div align="right" style="color: #978282; line-height: 0; font-size: 2vmin">
                    <em>
                      See <a href="https://en.wikipedia.org/wiki/VT220">vt220</a>
                    </em>
                  </div>
                  <figcaption>
                    <div style="background-color: red; margin-right:auto; margin-left:auto; text-align: center;">
                       <i style="color: white">B: Terminal 2</i>
                    </div>
                  </figcaption>
                </figure>
            </div>
        </td>
    </tr>
    <caption align="bottom" style="text-align: left; padding: 0; margin: 0;" >
          <i>Figure 1: An <a href="https://en.wikipedia.org/wiki/VT220">vt220</a> ASCII Temrinal</i> 
    </caption>
</table>
Here is a single figure that spans that text wraps to left around.
In the <a href="#fig:1Term-2">figure</a> we see a classic ASCII terminal.

<table id="fig:3Terms" align="left" width="50%" cellpadding="0" cellspacing="0" border="0" style="border-collapse: collapse; margin: auto 2em auto auto" >
    <tr style="padding: 0; margin: 0;">
        <td style="padding: 0; margin: 0; background-color: white;">
            <div style="margin-right: auto; margin-left: auto; padding: 0; margin: 0;">
            <figure style="width:100%; padding: 0; margin: 0;border: 1px solid red;" >
                <img src="underthecovers/images/shell.jpg" width="100%" style="padding: 0; margin: 0;">
                <figcaption>
                    <div style="background-color: red; margin-right:auto; margin-left:auto; text-align: center;">
                       <i style="color: white">B: Terminal 2</i>
                    </div>
                </figcaption>
            </figure>
            </div>
        </td>
        <td style="padding: 0; margin: 0; background-color: white;">
            <div style="margin-right: auto; margin-left: auto; padding: 0; margin: 0;">
            <figure style="width:100%;  padding: 0; margin: 0; border: 1px solid green;">
                <img src="underthecovers/images/shell.jpg" width="100%"style="padding: 0; margin: 0;">
                <figcaption> 
                    <div style="background-color: green; margin-right:auto; margin-left:auto; text-align: center;">
                        <i style="color: white">B: Terminal 2</i>
                    </div>
                </figcaption>
            </figure>
            </div>
        </td>
    </tr>
    <tr style="padding: 0; margin: 0;">
         <td colspan="2" width="50%" style="background-color: white; padding: 0; margin: 0;" >
           <div style="margin-right: auto; margin-left: auto;">
              <figure style="margin-right: auto; margin-left: auto;  width:50%; border: 1px solid blue;">
                  <img src="underthecovers/images/shell.jpg" align="center" width="100%">
                  <figcaption> 
                      <div style="background-color: blue; margin-right:auto; margin-left:auto; text-align: center;">
                          <i style="color: white">A: Terminal 3</i>
                      </div>
                  </figcaption>
              </figure>
            </div>
         </td>
    </tr>
    <caption align="bottom" style="text-align: left; padding: 0; margin: 0;" >
        <i>Figure 4: 3 Terminals</i>
    </caption>
</table>

This is a more complex example where we layout three images with a custom layout.  This <a href="#fig:3Terms">figure</a> has sperate captions for each sub figure.  Alignment allows left, right and center.  Left and right permit text wrapping around the figure.

<a id='htmtFig'></a>

# htmlFig

Like above but using htmlFig code to generate the html automatically.  In this case 3 figure in the top row and one in the bottom. 

htmFig provides an interface for generating html figures that use tables to permit complex layouts. 
However, the interface is designed to permit simple use cases and progressively more complex ones with little effort.
The following demonstrates a progressive series of uses to get to the equivalent of the above example

## Version 1: One image no styling or captions.

A string file path is assumed to be an image file that will be used to generate the figure.
We passing in a width to set the global size of the complete figure to be 25% of the page width. 
If we omitted this the default is 100%.  Note the function returns plain text html.  
To display it from a code cell you will need to wrap the result in calls to `HTML()` or `Markdown()` and then
`display()`.  I find `Markdown()` to be more flexible so that extra output can be markdown and styled appropriately.   Eg. the below output was generate by
```
display(Markdown(htmlFig("underthecovers/images/shell.jpg",width="25%")))
```

In [ ]:
display(Markdown(htmlFig("underthecovers/images/shell.jpg",width="25%")))

## Version 2:  images on two rows no styling or captions

By passing in a list of file paths we can get rows of images in our figure.

In [ ]:
display(Markdown(htmlFig(["underthecovers/images/shell.jpg",
                      "underthecovers/images/shell.jpg"],
                     width="25%"
                     )))

## Version 3:  Adding column of images on two rows no styling or captions

By passing in a list for a row specifies that the row should have multiple columns

In [ ]:
display(Markdown(htmlFig([
    # first row
    ["underthecovers/images/shell.jpg",
     "underthecovers/images/shell.jpg",
     "underthecovers/images/shell.jpg"],
    # second row
    "underthecovers/images/shell.jpg"
], width="50%")))

## Version 4:  Provide styling for an img 

Image specific parameters can be passed by using an dictionary rather than just a file path 
for an image.  In this case you specify key and values.  For the file path the key is 'src'.
The following documents many of the keys that you might want to override.  There are more but
I don't have time to document them now.

In [ ]:
display(HTML(htmlFig([
    # first row
    ["underthecovers/images/shell.jpg",
     "underthecovers/images/shell.jpg",
     "underthecovers/images/shell.jpg"],
    # second row
    {'src':"underthecovers/images/shell.jpg",
     # only one figure in this row colspan = 3
     'colspan': '3',
     # since colspan > 1 we overide the figure width 
     'figwidth': '33.3%',
     # add colored border
     'border': '1px solid red',
     # add subfigure specific caption
     'caption': 'C: Terminal 3',
     # override color of font and background for caption
     'capcolor': 'white',
     'capbgcolor': 'red'
    }   
], width="50%")))

## Version 5:  More per image customization and a global settings

Ok lets get things looking nice for all the images now by adding dictionaries for each image.  We also  add a caption for the entire figure, align it to the left (default is center) so that 
text below it will wrap to the right. So we also override the margins to give us 1 font width of
spacing on the right.

In [ ]:
display(Markdown(htmlFig([
    # first row
    [
      {'src':"underthecovers/images/shell.jpg",
       'border': '1px solid green',
       'caption': 'A: Terminal 1',
       'capcolor': 'white',
       'capbgcolor': 'green'
      },
      {'src':"underthecovers/images/shell.jpg",
       'border': '1px solid blue',
       'caption': 'B: Terminal 2',
       'capcolor': 'white',
       'capbgcolor': 'blue'
      },
      {'src':"underthecovers/images/shell.jpg",
       'border': '1px solid orange',
       'caption': 'C: Terminal 3',
       'capcolor': 'white',
       'capbgcolor': 'orange'
      }
    ],
    # second row
    {'src':"underthecovers/images/shell.jpg",
     'colspan': '3',
     'figwidth': '33.3%',
     'cellmargin': '0 auto 0 auto',
     'figmargin': '0 auto 0 auto',
     'divmargin': '0 auto 0 auto',
     'border': '1px solid red',
     'caption': 'C: Terminal 4',
     'capcolor': 'white',
     'capbgcolor': 'red'
    }   
], 
    caption="Figure: A bunch of VT220 Terminals",
    align="left",
    margin="auto 1em auto auto",
    width="50%") + '''
Now this looks nice!  Last thing to do is add an id so that 
we can have references to the figure.  See the last version below.
''' 
))

## Version 6:  Finally we can add an id so that we can reference the figure from other locations


In [ ]:
display(Markdown(htmlFig(id="fig:4Terms",
    imgs=[
    # first row
    [
      {'src':"underthecovers/images/shell.jpg",
       'border': '1px solid green',
       'caption': 'A: Terminal 1',
       'capcolor': 'white',
       'capbgcolor': 'green'
      },
      {'src':"underthecovers/images/shell.jpg",
       'border': '1px solid blue',
       'caption': 'B: Terminal 2',
       'capcolor': 'white',
       'capbgcolor': 'blue'
      },
      {'src':"underthecovers/images/shell.jpg",
       'border': '1px solid orange',
       'caption': 'C: Terminal 3',
       'capcolor': 'white',
       'capbgcolor': 'orange'
      }
    ],
    # second row
    {'src':"underthecovers/images/shell.jpg",
     'colspan': '3',
     'figwidth': '33.3%',
     'border': '1px solid red',
     'cellmargin': '0 auto 0 auto',
     'figmargin': '0 auto 0 auto',
     'divmargin': '0 auto 0 auto',
     'caption': 'C: Terminal 4',
     'capcolor': 'white',
     'capbgcolor': 'red'
    }   
], 
    caption="Figure: A bunch of VT220 Terminals",
    align="left",
    margin="auto 1em auto auto",
    width="50%") + '''
By adding the id field we can now reference the figure using the id in the normal way
<a href="#fig:4Terms">figure of 4 Terminals</a>.  Go <a href="#farawayref">here</a> to test coming back via 
the id.  Remember the reference is done by adding an html anchor tag with an href to be the id prefixed with the '#' 
Eg. `<a href="#fig:4Terms">figure of 4 Terminals</a>`
''' 
))

## htmlFig different sized imgs on the same row

When you need to have images of scale images on the same row you will want to override the cellwidth not the figwidth like the example below.   Use figwidth when you have a cell that is spanning multip columns as the above example.

In [ ]:
print(htmlFig(
    [[
        {'src':"underthecovers/images/UnixL01_SHCHT/041SHLLChat.png",
         'cellwidth': '47.5%'},
        {'src':"underthecovers/images/terminalwins.png",
         'cellwidth': '52%'}
     ]],
    caption="Figure: Today terminal emulators running on our desktop act act as ASCII terminals. Every new window is a new terminal connection."
))

This example uses div instead of the table tags.   As far as I can tell this is becoming the preferred way of doing it but it seems to have more css dependencies and for the moment the python code I have written htmFig uses tables.  Maybe in the future will switch to using divs to simplify the html produced.  But not today.

<div class="row">
     <div class="column" style="float: left; width: 33.33%; padding: 2px;">
       <img src="underthecovers/images/UnixL01_SHCHT/06SHLLChat.png" style="width:100%">
     </div>
     <div class="column" style="float: left; width: 33.33%; padding: 2px;">
        <img src="underthecovers/images/UnixL01_SHCHT/07SHLLChat.png" style="width:100%">
     </div>
     <div class="column" style=" float: left; width: 33.33%; padding: 2px;">
       <img src="underthecovers/images/UnixL01_SHCHT/08SHLLChat.png"  style="width:100%">
    </div>
</div>

Can't see this in the note book but there is some html here that causes two images to overlap
<div style="float:right">
<svg width="338" height="104">
  <clipPath id="myContainer">
    <rect width="338" height="104"></rect>
  </clipPath>
  <image width="338" height="104" xlink:href="https://www.edoardovignati.it/wp-content/uploads/2021/06/overlap-sunset.png" clip-path="url(#myContainer)"></image>
  <image width="338" height="104" xlink:href="https://www.edoardovignati.it/wp-content/uploads/2021/06/overlap-mountain.png" clip-path="url(#myContainer)"></image>
</svg>
</div>

<a id='displayBytes'></a>

# displayBytes

displayBytes is some python code to make displaying binary bitwise data easier and flexible.

The following are some examples of its use.


In [ ]:
X=np.uint16(0x8fff)
#count=16
#np.unpackbits(np.array([X], dtype=">i4").view(np.uint8))
#np.unpackbits(np.array([X], dtype=">i"+str(np.int8(count/8))).view(np.uint8))
displayBytes([X],numbits=16,columns=bitLabels(16), labels='')

In [ ]:
# as simple as it gets
displayBytes([0xff],labels='')

In [ ]:
# no headers
displayBytes([0xff],columns=[])

In [ ]:
# row labels
displayBytes([0xff],labels=["ALL ON"])

In [ ]:
# no column labes but row labels
# displayBytes([0xff],columns=[],labels=["ALL ON"])

In [ ]:
# how one might use display bytes to illustrate a problem 
# and solution
# create some binary varaible to use in our examples
# We use numpy types so that we get familar C/assembly byte types
u=np.uint8(0x55)
v=np.uint8(0xaa)
w=np.bitwise_and(u,v)

# Problem uses a blank "answer" row
displayBytes([[u],[v],[""]],labels=["u","v", "u & v"], td_height="85px")


In [ ]:
# updated to include answer
displayBytes([[u],[v],[w]],labels=["u","v", "u & v"])

In [ ]:
# something fancier that uses the ability to display other bit widths
X=np.uint8(0xae)
XL=np.bitwise_and(X,0xf)
XH=np.bitwise_and(np.right_shift(X,4),0xf)
displayBytes(bytes=[[X]])

In [ ]:
displayBytes(bytes=[[XH]], numbits=4,columns=["[$b_7$", "$b_6$", "$b_5$", "$b_4$]"])

In [ ]:
displayBytes(bytes=[[XL]], numbits=4,columns=["[$b_3$", "$b_2$", "$b_1$", "$b_0$]"])

In [ ]:
displayStr(format(XH,"1X"), size="2em", align="center")

In [ ]:
displayStr(format(XL,"1X"), size="2em", align="center")

In [ ]:
displayStr("0x"+format(X,"02X"), size="2em", align="center")

In [ ]:
# Example of using displayBytes to create an hex table
displayBytes(bytes=[[i] for i in range(16)],td_font_size="1.5vw", th_font_size="1vw", numbits=4, columns=["[$b_3$", "$b_2$", "$b_1$", "$b_0$]"], labels=[format(i,"1X")for i in range(16)], labelstitle="HEX", center=True)

In [ ]:
# an quick and dirty table of all byte values
displayBytes(bytes=[[i] for i in range(256)], labelstitle="HEX (DEC)", labels=["0x"+format(i,"02x")+ " (" + format(i,"03d") +")" for i in range(256)], center=True)

In [ ]:
# quick and dirty ascii table
displayBytes(bytes=[[i] for i in range(128)], labels=["0x"+format(i,"02x")+ " (" + format(i,"03d") +") ASCII: " + repr(chr(i)) for i in range(128)])

In [ ]:
display(Markdown('''# Experiment to see how to get another header above a set of colomns

No support in displayBytes for this yet.
'''))
cols=["C", "B", "A"]
cols=list(zip(['BINARY']*3,cols))
cols = pd.MultiIndex.from_tuples(cols, names=[' ', 'HEX']) 
df = pd.DataFrame([[1, 2, 3],[4, 5, 6],[7,8,9]],index=["r1", "r2", "r3"], columns=cols)
#df.rename_axis("hex")
df

<a id="TermShellCmd"></a>

# TermShellCmd

In [ ]:
TermShellCmd("ls")

In [ ]:
TermShellCmd("stty -a")

In [ ]:
TermShellCmd("foobarblah one two three")

<a id='TtySession'></a>

## TtySession, bashSession and the BashSession Class 

Simple examples: Starts a new shell for the commands and then kills the Shell

In [ ]:
bashCmds("ls")

In [ ]:
bashCmds("pwd\nls")

In [ ]:
bashCmds("help", height="10em")

We can also create a persistent session and then can reuse it to run commands.  Important point is that the same shell process will be kept alive

In [ ]:
session = bashSessionOpen()
#print(session)

In [ ]:
bashCmds("pwd\nls\necho $x", session=session, close=False)

In [ ]:
bashCmds("x=foo", session=session, close=False)

In [ ]:
bashCmds("echo $x", session=session, close=False)

You can render all the accumulated output from a session

In [ ]:
renderTtySessionOutput(session)

You can also use the raw write interface to send a raw bytes in stead of command lines to the bash process.  By default bytes will be sent one at a time with a 140 ms delay between characters.  After the data is sent and EOF will be generated.  All these defaults can be over ridden as needed.  See the code 

In [ ]:
output, session = bashSessionRawWrite('''cat
hello
''', session, 1)
renderTtySessionOutput(session) 

In [ ]:
bashCmds("date", session=session, close=False)

To clean up a session -- kill the associated process

In [ ]:
print(TtyOpenSessions)
closeTtySession(session)
print(TtyOpenSessions)

Cleanup all sessions that have been created

In [ ]:
closeAllOpenTtySessions()
print(TtyOpenSessions)

### The BashSession Class

The BashSession make it easier to create and use sessions

In [ ]:
bs = BashSession()

In [ ]:
bs.run("ls")

In [ ]:
bs.run("y=2\n((y++))")

In [ ]:
bs.run("echo $y")

In [ ]:
bs.output()

Or we can get running output

In [ ]:
bs.runAllOutput("echo 'foo'")

In [ ]:
bs.runNoOutput("date")

In [ ]:
bs.output()

In [ ]:
bs.rawWrite('''cat
hello
''')

Close the session

In [ ]:
del bs

We can use raw writes incrementally to interact with a program such as `cat`

In [ ]:
cat = BashSession()
cat.rawWriteAllOutput('''cat
''', sendEOF=False)

In [ ]:
cat.rawWrite('''hello
''', sendEOF=False)

In [ ]:
cat.rawWrite('''goodbye
''', sendEOF=False)

To send EOF and gather remaining output write a empty string

In [ ]:
cat.rawWrite('')

In [ ]:
cat.output()

In [ ]:
del cat

In [ ]:
print(TtyOpenSessions)

In [ ]:
closeAllOpenTtySessions()

<a id="MDBox"></a>
# MDBox, FileMDBox, FileCodeBox

# Raw Markdown and html

<div style="width:100%; height:200px; overflow: auto;" >

#### lots of little programs that naturally feel like built-ins
- `echo $PATH`
   - `ls /bin`
      - see `ls` there?
      - notice `bash` ;-)
   - how can we figure out where `ls` is coming from
      - bash has handy built in called `type`
        - `help type`
        - `type -a ls`
   - by tradition all preinstalled programs should have a manual page
        - `man ls`
        - `man bash`
        - `man <something in /bin>`

## extendent the shell is easy
- set `PATH="$HOME/bin:$PATH`
  - putting a program or script called `foobar` in `$HOME/bin`
    - now `$ foobar` will feel like a built-in

## overide what's there
- putting `ls` in `$HOME/bin` will now overide the others
- Even override a built in
  - `function echo() { builtin echo -n "myecho: "; builtin echo $@ }`
  - will talk about this one later

## natural ways of composing and extending via programming
Two important ones:

1. shell scripts
2. command pipelines

### shell provide natural model for composition

- put shell commands in a file : eg put this in `foobar`
``` bash
#!/bin/bash
echo "My first shell script"
echo "foobar"
```
- mark as executable
- now shell will be able to run `foobar`

### pipeline: allow programs to be easily composed

- `ls -1` list files - one per line
- `wc -l` counts lines
- `ls -l | wc -l` - tells us how man files in this direcotyr
- `ls -l /bin | wc -l`
-  or to get really fancy
  - exploit more knowledge about shell expansion abilities
  - `echo $PATH`
  - `echo ${PATH//:/ }`
  - `ls -1 ${PATH//:/ } | wc -l`
     - what do you think this did?  
</div>

# using a markdown file

In [ ]:
display(Markdown(FileMDBox(title="# Intro to the Book", file="underthecovers/intro_tb.md", w="100%", h="200px")))

### In line markdown

In [ ]:
display(Markdown(MDBox(w="100%", h="200px",
                      contents='''
#### lots of little programs that naturally feel like built-ins
- `echo $PATH`
   - `ls /bin`
      - see `ls` there?
      - notice `bash` ;-)
   - how can we figure out where `ls` is coming from
      - bash has handy built in called `type`
        - `help type`
        - `type -a ls`
   - by tradition all preinstalled programs should have a manual page
        - `man ls`
        - `man bash`
        - `man <something in /bin>`

## extendent the shell is easy
- set `PATH="$HOME/bin:$PATH`
  - putting a program or script called `foobar` in `$HOME/bin`
    - now `$ foobar` will feel like a built-in

## overide what's there
- putting `ls` in `$HOME/bin` will now overide the others
- Even override a built in
  - `function echo() { builtin echo -n "myecho: "; builtin echo $@ }`
  - will talk about this one later

## natural ways of composing and extending via programming
Two important ones:

1. shell scripts
2. command pipelines

### shell provide natural model for composition

- put shell commands in a file : eg put this in `foobar`
``` bash
#!/bin/bash
echo "My first shell script"
echo "foobar"
```
- mark as executable
- now shell will be able to run `foobar`

### pipeline: allow programs to be easily composed

- `ls -1` list files - one per line
- `wc -l` counts lines
- `ls -l | wc -l` - tells us how man files in this direcotyr
- `ls -l /bin | wc -l`
-  or to get really fancy
  - exploit more knowledge about shell expansion abilities
  - `echo $PATH`
  - `echo ${PATH//:/ }`
  - `ls -1 ${PATH//:/ } | wc -l`
     - what do you think this did?                        
                      '''
                      )))

### A version to make displaying code FileCodeBox (uses MDBox) 

In [ ]:
display(Markdown(FileCodeBox(
    file="underthecovers/src/shlldirfile.sh", 
    lang="bash", 
    title='# File and Directories - Part I',
    w="100%",
    h="200px")))

# Rough notes and playing around

Operate first url to directly spawn bu image

https://jupyterhub-opf-jupyterhub.apps.zero.massopen.cloud/hub/spawn?bu-cs-jupyter-book:latest

Working with Tom on operate first slack the following url git clone 


https://jupyterhub-opf-jupyterhub.apps.zero.massopen.cloud/hub/user-redirect/git-pull?bu-cs-jupyter-book%3Alatest=&repo=https%3A%2F%2Fgithub.com%2Fjappavoo%2FUndertheCovers&urlpath=tree%2FUndertheCovers%2Funderthecovers%2FL00_210_JA.ipynb&branch=main


How to set 
```
c.Spawner.default_url = '/data/jupyterhub/jupyterhub_notebooks'
c.Spawner.notebook_dir = '/data/jupyterhub/jupyterhub_notebooks'
```

Hmmm it now seems that we are launching bu-cs container and home dir ends up being joyvan need to look at start up script as there is a lot of logic in it to adjust user and group

At this point the running joyvan user on operatefirst seems to be some system assigned id with group root which leads to not having the right permissions to write home dir

For the moment a hack might be to make the joyvan homdir and subdirs group root


Have managed to get the notebook running and now stuggling to get embedded terminals work.

1) calling the api to create the terminals works after getting the api token from the environment variables
2) however running into security setting need to be fixed
  ```
  
  c.Spawner.args = ['--NotebookApp.tornado_settings={"headers":{"Content-Security-Policy": "frame-ancestors * self host_ip:port"}}']   

c.JupyterHub.tornado_settings = { 'headers': { 'Content-Security-Policy': "frame-ancestors * self host_ip:port"} }
```
see: 
- https://github.com/jupyterhub/jupyterhub/issues/379#issuecomment-353607675
- https://github.com/jupyterhub/the-littlest-jupyterhub/issues/312#issuecomment-590679516


In [ ]:
print(base_url)

In [ ]:
showET()

In [ ]:
a = "foo"
a += " bar"
print(a)
img = {'src':"file.jpg", 'blah' : 'a b cm'}

if img.get('src'):
    print(img.get('src'))
    
#if img.get('blah'):
blah=img.get('blah')
if blah:
    print(blah)

In [ ]:
%run -i underthecovers/python/common.py

In [ ]:
a=12

what is the value{{a}}mean?

In [ ]:
html=htmlFig(
        imgs=[
            [
                {
                'src' :"underthecovers/images/shell.jpg", 
                'border':"1px solid red",
                'caption': "A: Terminal 1",
                'capcolor': 'white',
                'capbgcolor': 'red',
                'extratxt': 'See <a href="https://en.wikipedia.org/wiki/VT220">vt220</a>',
                'extrafont': "1vmin",
                'extracolor': "green"
                }, 
                {
                'src':"underthecovers/images/shell.jpg", 
                'border':"1px solid green",
                'caption': "B: Terminal 2",
                'capcolor': 'white',
                'capbgcolor': 'green',
                },
                {
                'src':"underthecovers/images/shell.jpg", 
                'border':"1px solid orange",
                'caption': "C: Terminal 3",
                'capcolor': 'white',
                'capbgcolor': 'orange',
                }
            ],
            { 
                'src': 'underthecovers/images/shell.jpg',
                'border':"1px solid blue",
                'caption': "D: Terminal 4",
                'capcolor': 'white',
                'capbgcolor': 'blue',
                'colspan': '3',
                'width': '33.3%'
            }
        ], 
        align="left",
        width="50%",
       caption='<center>Figure 1: Some <a href="https://en.wikipedia.org/wiki/VT220">vt220</a> ASCII Terminals</center>',
#        subcaption="B: Terminal 2", 
#        extratxt=,
#        border="1px solid red"
    )
display(HTML(html))
#print(html)

This is some text

In [ ]:
## def toImg(i):
    # if a string the convert to img
    if (type(i) == type("")):
        i={"src":i}

    if ((not type(i) == type({})) or (not 'src' in i)):
        raise ValueError('img must at have a src specified')
        
    return i

# a list of imgs
def toImgs(i):
    # already a list so don't do anyting
    if type(i) == type([]):
        return i
    
    i=[toImg(i)]
    
    return i

def testsrc(imgs):
    imgs=toImgs(imgs)
    rows = len(imgs)
    maxcols = 1

    # calculate the maximum number of columns
    # and build new list
    rows = []
    for r in imgs:
        r = toImgs(r)
        rows.append(r)
        if (len(r)>maxcols):
            maxcols = len(r);
    
    for r in rows:
        cols=len(r)
        print("row: " + str(cols) + "/" + str(maxcols))
        for i in r:
            print("cells:");
            img=toImg(i)
            print(img)
    
testsrc(imgs=[[{'src':"file.jpg",'caption':"a picture"},"file.jpg"],"file.jpg"])

https://jupyterhub.github.io/nbgitpuller/link?hub=https://127.0.0.1

<img src="underthecovers/images/SLS_WPIM.svg" width="100%" style="vertical-align:top; float:middle; border:0px; margin: 10px 0px; background-color: #191919;">
<img src="underthecovers/images/SLS_TheMachine.svg" width="100%" style="vertical-align:top; float:middle; border:0px; margin: 0px 0px;">

In the <a href="#vt220Fig">figure</a> we see a classic ASCII terminal.

```{glossary}
Term one
  An indented explanation of term 1

A second term
  An indented explanation of term2
```

<div class="admonition note" name="html-admonition" style="background: lightgreen; padding: 10px">
<p class="title">This is the **title**</p>
This is the *content*
</div>

<p id="farawayref">
testing a reference to a figure up in the page <a href="#fig:4Terms">Four Terminal Figure</a>
    </p>

In [ ]:
import pexpect

In [ ]:
shell.close()
shell = pexpect.spawn('/bin/bash -l -i')

In [ ]:
print(shell)

In [ ]:
shell.sendline()
shell.expect('\r\n$ ')
print(shell.before)
print(shell.after)

In [ ]:
print(shell)

In [ ]:
shell.close()

In [ ]:
print(shell)

In [ ]:
from IPython.display import display, update_display

In [ ]:
MIMETYPE = "application/x-drawio"

In [ ]:
def display_drawio(xml, **kwargs):
    """ Send some xml to the frontend
    
    Get a handle to update later by calling `display_drawio` with `display_id=True` 
    """
    return display({MIMETYPE: xml}, raw=True, **kwargs)

In [ ]:
!ls
handle = display_drawio("/Work/UndertheCovers/underthecovers/images/src/BashProcessExecution.drawio")

In [ ]:
from ipydrawio_widgets import Diagram

diagram = Diagram()
diagram